In [ ]:
import os
import urllib.request
from os import path
from pathlib import Path

import numpy as np
from PIL import Image

from myoquant.common_func import (
    load_cellpose,
    load_sdh_model,
    load_stardist,
    run_cellpose,
    run_stardist,
)
from myoquant.HE_analysis import run_he_analysis
from myoquant.SDH_analysis import run_sdh_analysis

try:
    from imageio.v2 import imread
except:
    from imageio import imread

In [ ]:
# ANALYSE SDH

# Importer l'image et dossier de sauvegarde
image_path = "LE CHEMIN VERS L'IMAGE SDH A ANALYSER"
output_path = image_path.parents[0]
Path(output_path).mkdir(parents=True, exist_ok=True)

# Télécharger le modèle SDH
model_path_abs = Path(os.path.abspath(__file__)).parents[0] / "model.h5"
if not path.exists(model_path_abs):
    urllib.request.urlretrieve(
        "https://huggingface.co/corentinm7/MyoQuant-SDH-Model/resolve/main/model.h5",
        model_path_abs,
    )
model_path = model_path_abs

# Charger Cellpose
model_cellpose = load_cellpose()
# Charger l'image
image_ndarray_sdh = imread(image_path)

# Faire tourner CellPose sur l'image & sauvegarder
mask_cellpose = run_cellpose(image_ndarray_sdh, model_cellpose)
mask_cellpose = mask_cellpose.astype(np.uint16)
cellpose_mask_filename = image_path.stem + "_cellpose_mask.tiff"
Image.fromarray(mask_cellpose).save(output_path / cellpose_mask_filename)

# Faire tourner le modèle SDH sur l'image et récupérer le tableau de sommaire (result_df)
model_SDH = load_sdh_model(model_path)
result_df, full_label_map = run_sdh_analysis(
    image_ndarray_sdh, model_SDH, mask_cellpose
)
csv_name = image_path.stem + "_results.csv"
result_df.to_csv(
    output_path / csv_name,
    index=False,
)
label_map_name = image_path.stem + "_label_map.tiff"
Image.fromarray(full_label_map).save(output_path / label_map_name)

In [ ]:
# ANALYSE HE

# Importer l'image et dossier de sauvegarde
image_path = "LE CHEMIN VERS L'IMAGE HE A ANALYSER"
output_path = image_path.parents[0]
Path(output_path).mkdir(parents=True, exist_ok=True)

# Charger les modèles CellPose et Stardist
model_cellpose = load_cellpose()
model_stardist = load_stardist()
# Charger l'image
image_ndarray = imread(image_path)
# Faire tourner Cellpose (fibres) puis sauvegarder l'image
mask_cellpose = run_cellpose(image_ndarray, model_cellpose)
mask_cellpose = mask_cellpose.astype(np.uint16)
cellpose_mask_filename = image_path.stem + "_cellpose_mask.tiff"
Image.fromarray(mask_cellpose).save(output_path / cellpose_mask_filename)
# Faire tourner Stardist (noyaux) puis sauvegarder l'image
mask_stardist = run_stardist(image_ndarray, model_stardist)
mask_stardist = mask_stardist.astype(np.uint16)
stardist_mask_filename = image_path.stem + "_stardist_mask.tiff"
Image.fromarray(mask_stardist).save(output_path / stardist_mask_filename)
# Analyser la position des noyaux et fibres et sauvegarder le tableau de sommaire (result_df)
result_df, full_label_map = run_he_analysis(image_ndarray, mask_cellpose, mask_stardist)
csv_name = image_path.stem + "_results.csv"
result_df.to_csv(
    output_path / csv_name,
    index=False,
)
label_map_name = image_path.stem + "_label_map.tiff"
Image.fromarray(full_label_map).save(output_path / label_map_name)